In [1]:
import pandas as pd

corpus = pd.read_parquet("hf://datasets/MarkrAI/triviaqa_sample_autorag/corpus.parquet")
corpus.head()

,doc_id,contents,metadata
0,tc_2_4,Alvin and the Chipmunks (2007) - IMDb\nIMDb\n1...,{'last_modified_datetime': 2024-02-04 17:15:47...
1,tc_2_6,The Chipmunks - Biography | Billboard\nThe Chi...,{'last_modified_datetime': 2024-02-04 17:15:47...
2,tc_2_9,Alvin and the Chipmunks: The Squeakquel – EW.c...,{'last_modified_datetime': 2024-02-04 17:15:47...
3,tc_2_16,Alvin Seville | Alvin and the Chipmunks Wiki |...,{'last_modified_datetime': 2024-02-04 17:15:47...
4,tc_2_21,A Christmas Story | Christmas Specials Wiki | ...,{'last_modified_datetime': 2024-02-04 17:15:47...


In [2]:
splits = {'train': 'qa_train.parquet', 'test': 'qa_test.parquet'}
df = pd.read_parquet("hf://datasets/MarkrAI/triviaqa_sample_autorag/" + splits["train"])
df.head()

,qid,query,generation_gt,retrieval_gt
0,a5a9ca54-c0f6-45fa-8475-c33f3e351044,Who was the man behind The Chipmunks?,[david seville],"[tc_2_4, tc_2_6, tc_2_9, tc_2_16, tc_2_21, tc_..."
1,78a98148-c4fc-4cde-933b-ddbfd8a4a8e0,What star sign is Jamie Lee Curtis?,"[scorpio, scorpio disambiguation, skorpio]",[tc_13_0]
2,27229cd3-dd29-4c3e-b715-aa525e1dba86,Which Lloyd Webber musical premiered in the US...,"[sunset bulevard, sunset blvd, sunset boulevar...","[tc_33_0, tc_33_1, tc_33_2, tc_33_3, tc_33_5, ..."
3,43487767-a02e-42ea-a48b-c5c0a1c2983c,Who was the next British Prime Minister after ...,"[henry campbell bannerman, campbell bannerman,...","[tc_40_0, tc_40_1, tc_40_2, tc_40_3, tc_40_4, ..."
4,647f67ba-ea61-4023-973a-fed91f98def0,Who had a 70s No 1 hit with Kiss You All Over?,"[transported for life, self exile, exile and b...","[tc_49_1, tc_49_6, tc_49_8, tc_49_20, tc_49_21..."


In [3]:
# Explode the `retrieval_gt` column
df_exploded = df.explode('retrieval_gt')

# Merge with `corpus` on document ID
merged_df = pd.merge(df_exploded, corpus, left_on='retrieval_gt', right_on='doc_id', how='left')

# Group by `qid` and aggregate the `contents`
grouped_df = merged_df.groupby('qid')['contents'].apply(lambda x: ' '.join(x)).reset_index()

# Merge back with the original `df` to get the final result
final_df = pd.merge(df, grouped_df, on='qid', how='left')

In [4]:
final_df = final_df[['query', 'generation_gt', 'contents']]

In [5]:
for i in range(len(final_df)):
    final_df.loc[i, "generation_gt"] = final_df.loc[i, 'generation_gt'][0]

final_df.head()

,query,generation_gt,contents
0,Who was the man behind The Chipmunks?,david seville,Alvin and the Chipmunks (2007) - IMDb\nIMDb\n1...
1,What star sign is Jamie Lee Curtis?,scorpio,Jamie Lee Curtis - IMDb\nIMDb\n17 January 2017...
2,Which Lloyd Webber musical premiered in the US...,sunset bulevard,Andrew Lloyd Webber | The official website for...
3,Who was the next British Prime Minister after ...,henry campbell bannerman,"The Balfour Declaration, 1917 (History and Ful..."
4,Who had a 70s No 1 hit with Kiss You All Over?,transported for life,Exile - I Wanna Kiss You All Over (1978) - You...


In [6]:
import torch

torch.cuda.empty_cache()

In [7]:
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Meta-Llama-3.1-8B-Instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct", use_fast=False)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 10,
    "return_full_text": False,
    "do_sample": False,
}

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [14]:
def produce_prompt(question, correct):

    user_content = "Question: "+question+"\n Correct answer: "+correct+"\n\n Incorrect answer:"
    messages = [ 
        {"role": "system", "content": """
        You are a helpful AI assistant. You are given a question and the correct answer to it.
        You have to provide me an alternative, wrong answer to it. 
        """}, 
        {"role": "user", "content": user_content},
    ] 
    return messages

In [17]:
for i in range(10):
    output = pipe(produce_prompt(final_df['query'][i], final_df['generation_gt'][i]), **generation_args) 
    print(output[0]['generated_text']) 

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Ross Weathersby


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Jamie Lee Curtis is a Gemini.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

The incorrect answer is: Moonraker film.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Twenty million.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The incorrect answer is: The state of Hawaii.
Marilyn Monroe.
